# Building and Training a Custom ConvLogic Model for MNIST

## 🔧 Clone & Install ConvLogic Repository

We begin by cloning the `convlogic` GitHub repository and installing all necessary dependencies.

In [2]:
!git clone https://github.com/lkorinek/convlogic.git

Cloning into 'convlogic'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 97 (delta 20), reused 90 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 557.12 KiB | 9.61 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [3]:
!pip install -r convlogic/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 114.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [4]:
!pip install -v convlogic/. --no-deps --no-build-isolation

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Processing ./convlogic
  Running command Preparing metadata (pyproject.toml)
  >>> Building with CUDA support
  running dist_info
  creating /tmp/pip-modern-metadata-0oyyi325/convlogic.egg-info
  writing /tmp/pip-modern-metadata-0oyyi325/convlogic.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-modern-metadata-0oyyi325/convlogic.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-modern-metadata-0oyyi325/convlogic.egg-info/requires.txt
  writing top-level names to /tmp/pip-modern-metadata-0oyyi325/convlogic.egg-info/top_level.txt
  writing manifest file '/tmp/pip-modern-metadata-0oyyi325/convlogic.egg-info/SOURCES.txt'
  /usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:576: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
    warnings.warn(msg.format('we could not find n

In [5]:
import sys
sys.path.append("/content/convlogic/src")

In [6]:
import os
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger, WandbLogger

from data import ConvLogicDataModule
from model import ConvLogicModel

from hydra import compose, initialize
from hydra.utils import get_original_cwd
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf

## ⚙️ Load and Customize Configuration

We use Hydra to load the base configuration and optionally override specific parameters such as the number of training epochs, learning rate, or dataset type. These values define the training behavior of the ConvLogic model.

In [7]:
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()
initialize(config_path="convlogic/configs", version_base="1.1")
cfg = compose(config_name="config")

# Override config if needed
cfg.trainer.max_epochs = 50
cfg.model.lr: 0.01
cfg.model.weight_decay: 0.0
cfg.model.k: 16
cfg.model.tau: 6.5
cfg.model.batch_size: 512
cfg.model.dataset_name = "mnist"
cfg.evaluate.model_filename = "model_mnist_demo"
cfg.general.seed = 42
print(OmegaConf.to_yaml(cfg))

general:
  seed: 42
  profile: false
  profile_type: advanced
logging:
  wandb: false
trainer:
  max_epochs: 50
  accelerator: gpu
  devices: 1
  deterministic: false
data:
  num_workers: 2
  threshold_levels: null
  train_val_split: 0.9
  data_dir: ../../data
early_stopping:
  use: false
  monitor: val_eval/acc
  min_delta: 0.001
  patience: 50
  mode: max
accuracy_threshold_stop:
  monitor: val_no_eval/acc
  use: false
  epoch: 50
  threshold: 0.58
evaluate:
  checkpoint_path: models/
  model_filename: model_mnist_demo
model:
  lr: 0.02
  weight_decay: 0.002
  k: 32
  tau: 20
  dataset_name: mnist
  batch_size: 128
  input_channels: null
  implementation: cuda



In [8]:
callbacks = ModelCheckpoint(
    monitor="val_eval/acc",
    mode="max",
    save_top_k=1,
    save_last=True,
    dirpath="models/",
    filename="model_cifar10_demo",
    auto_insert_metric_name=False,
)

## Prepare the MNIST Dataset

We use `ConvLogicDataModule` to load and preprocess the MNIST dataset. The data is thresholded to convert pixel values into binary inputs. We also extract the number of input channels from the dataset to configure our model.

In [9]:
dm = ConvLogicDataModule(
    dataset_name=cfg.model.dataset_name,
    data_dir=cfg.data.data_dir,
    batch_size=cfg.model.batch_size,
    num_workers=cfg.data.num_workers,
    threshold_levels=cfg.data.threshold_levels,
    train_val_split=cfg.data.train_val_split,
)
dm.prepare_data()
dm.setup("fit")

cfg.model.input_channels = dm.input_channels

100%|██████████| 9.91M/9.91M [00:00<00:00, 54.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.73MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 12.7MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.60MB/s]


## Define a Custom ConvLogic Model for MNIST

Here we define our own neural network using `ConvLogicLayer` for logic-based convolutions and `LogicLayer` for dense logic gate layers.

We integrate our custom `ConvLogicMnistModel` into the higher-level `ConvLogicModel` interface, which provides the training and evaluation logic.

In [13]:
from convlogic import ConvLogicLayer
from difflogic import LogicLayer, GroupSum

class ConvLogicMnistModel(nn.Module):
    """
    ConvLogic model for MNIST dataset.
    """

    def __init__(self, input_channels=1, input_size=28, k_param=32, tau=6.5, implementation="cuda"):
        super().__init__()
        c1, c2, c3 = k_param, 3 * k_param, 9 * k_param
        # First logic convolution layer with a larger kernel
        layers = [
            ConvLogicLayer(
                in_channels=input_channels,
                out_channels=c1,
                kernel=5,
                padding=0,
                residual_init=True, # Initializes weights to favor A gate (90%) (useful for training stability).
                complete=True, # The full 3-stage logic pipeline: ConvLogic → TreeLogic1 → TreeLogic2.
                implementation=implementation,
            ),
            ConvLogicLayer(
                in_channels=c1,
                out_channels=c2,
                kernel=3,
                padding=1,
                residual_init=True,
                complete=True,
                implementation=implementation,
            ),
            ConvLogicLayer(
                in_channels=c2,
                out_channels=c3,
                kernel=3,
                padding=1,
                residual_init=True,
                complete=True,
                implementation=implementation,
            ),
            # Flatten for fully connected layers
            nn.Flatten(),
            LogicLayer(81 * k_param, 1280 * k_param, residual_init=False, implementation=implementation),
            LogicLayer(1280 * k_param, 640 * k_param, residual_init=False, implementation=implementation),
            LogicLayer(640 * k_param, 320 * k_param, residual_init=False, implementation=implementation),
            GroupSum(k=10, tau=tau),
        ]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

model = ConvLogicModel(cfg.model)
# Replace the internal model with a custom-defined one
model.model = ConvLogicMnistModel(
    input_channels=cfg.model.input_channels,
    input_size=28,
    k_param=cfg.model.k,
    tau=cfg.model.tau,
    implementation=cfg.model.implementation
)

## Train the Model

Using PyTorch Lightning's `Trainer`, we now begin training our custom model. The training loop includes automatic checkpointing of the best-performing model based on validation accuracy.

In [15]:
trainer = Trainer(
    max_epochs=cfg.trainer.max_epochs,
    accelerator="gpu",
    devices=1,
    callbacks=callbacks,
    deterministic=cfg.trainer.deterministic,
)

trainer.fit(model, datamodule=dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                | Params | Mode 
----------------------------------------------------------
0 | model     | ConvLogicMnistModel | 1.2 M  | train
1 | criterion | CrossEntropyLoss    | 0      | train
----------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.774     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


## ✅ Evaluate on Test Set

After training, we evaluate the final model performance on the held-out MNIST test set.

- `test/acc_eval`: Accuracy using **discretized logic gates**, where each gate selects its most likely logic function. This reflects the performance of the model if deployed with fixed, hard logic.
- `test/acc`: Accuracy using **probabilistic logic functions**, where the output is a weighted sum over all possible gates. This reflects the soft version used during training.

In [16]:
trainer.test(model, datamodule=dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.9768629670143127     │
│       test/acc_eval       │    0.9773637652397156     │
│         test/loss         │    0.08079593628644943    │
│      test/loss_eval       │    0.08189473301172256    │
└───────────────────────────┴───────────────────────────┘

[{'test/loss_eval': 0.08189473301172256,
  'test/acc_eval': 0.9773637652397156,
  'test/loss': 0.08079593628644943,
  'test/acc': 0.9768629670143127}]